# IST597:- Multi-Layer Perceptron

## Load the libraries

In [40]:
import os
import numpy as np
import time
import tensorflow as tf
seed = 5308
np.random.seed(seed)
tf.random.set_seed(seed)

In [41]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [42]:
tf.executing_eagerly()

True

#Get number of Gpu's and id's in the system or else you can also use Nvidia-smi in command prompt.

## Generate random data

In [45]:
#Getting the datasets - mnist : digit dataset, fashion_mnist : labelled fashion images
#mnist = tf.keras.datasets.mnist
fashion_mnist = tf.keras.datasets.fashion_mnist

In [46]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [71]:
size_input = 784
size_hidden_1 = 256 
size_hidden_2 = 128
size_output = 10 #should be 10 since digits are from 0-9
number_of_train_examples = X_train.shape[0]
number_of_test_examples = X_test.shape[0]
print (f"{number_of_test_examples},{number_of_train_examples}")

10000,60000


In [ ]:
#Normalizing data
#X_train = X_train/255.0
#X_test = X_test/255.0

In [49]:
X_train = X_train.reshape(number_of_train_examples, -1)
X_test = X_test.reshape(number_of_test_examples, -1)
X_test.shape, X_train.shape

((10000, 784), (60000, 784))

In [50]:
#Normalizing data
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
y_train[0]

5

In [51]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [52]:
# Split dataset into batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(4)
train_ds,test_ds

(<BatchDataset shapes: ((None, 784), (None, 10)), types: (tf.float64, tf.float32)>,
 <BatchDataset shapes: ((None, 784), (None, 10)), types: (tf.float64, tf.float32)>)

# Build MLP using Eager Execution

## Combined model with L2 and dropout enable knobs

In [72]:
# Combined model with L2 and dropout enable knobs
import sys
from tensorflow.python.ops.gen_math_ops import square
import numpy as np
# Define class to build mlp model
class MLP_combined(object):
  def __init__(self, size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "default", lambda_l2 = 0.8, dropout = 0.3, lr = 0.05, device=None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    import sys
    self.size_input, self.size_hidden_1, self.size_hidden_2, self.size_output, self.reg_mode, self.lambda_l2, self.dropout, self.lr, self.device =\
    size_input, size_hidden_1, size_hidden_2, size_output, reg_mode, lambda_l2, dropout, lr, device
    
    # Initialize weights between input layer and hidden layer 1 - (128, 128)
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden_1]))
    # Initialize biases for hidden layer 1 - (1,128)
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden_1]))

    # Initialize weights between hidden layer 1 and hidden layer 2 - (128, 128)
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden_1, self.size_hidden_2]))
    # Initialize biases for hidden layer 2 - (1, 128)
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden_2]))

    # Initialize weights between hidden layer 2 and output layer - (128, 10)
    self.W3 = tf.Variable(tf.random.normal([self.size_hidden_2, self.size_output]))
    # Initialize biases for output layer - (1,10)
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))

    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
    
  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
  
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''

    #Use categorical cross entropy in place of MSE
    cce = tf.keras.losses.CategoricalCrossentropy()
    #loss = tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred)
    loss = cce(y_true, y_pred)

    if self.reg_mode == "L2":
      #Applying L2 regularization
      #print ("#Applying L2 regularization")
      para_list = [self.W1, self.W2, self.W3]
      regularizer = sum([tf.reduce_sum(tf.square(_var)) for _var in para_list])
      #loss = self.lambda_l2 * loss +  ((1 - self.lambda_l2)/ number_of_train_examples) * regularizer
      loss = loss +  ((self.lambda_l2/(2 * number_of_train_examples)) * regularizer)

    return loss
  
  def backward(self, X_train, y_train):
    """
    backward pass
    """
    optimizer = tf.keras.optimizers.SGD(learning_rate=self.lr)
    #optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
      grads = tape.gradient(current_loss, self.variables)
      optimizer.apply_gradients(zip(grads, self.variables))
        
        
  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    
    #Remember to normalize your dataset before moving forward
    
    # Compute values in hidden layer 1
    w1hat = tf.matmul(X_tf, self.W1) + self.b1
    h1hat = tf.nn.relu(w1hat)

    # Compute values in hidden layer 2
    w2hat = tf.matmul(h1hat, self.W2) + self.b2
    h2hat = tf.nn.relu(w2hat)

    if self.reg_mode == "dropout":
      # Adding dropout on layer 2
      #print ("# Adding dropout on layer 2")
      h2hat = tf.nn.dropout(h2hat, self.dropout)

    # Compute output
    output = tf.matmul(h2hat, self.W3) + self.b3
    #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
    #Second add tf.Softmax(output) and then return this variable
    output = tf.nn.softmax(output)
    #print (f"Output : {output}")
    return output

## Train Model

# Function to train a model and prints cross entropy and accuracy

In [73]:
# Function to train a model and prints cross entropy and accuracy
from tqdm import tqdm
def train_model(mlp, num_epochs, seed, shuffle_size, batch_size):
  time_start = time.time()
  for epoch in tqdm(range(0, num_epochs)):
    loss_total = tf.zeros([1,1], dtype=tf.float32)
    correct_preds_total = 0
    #lt = 0
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(shuffle_size, seed=epoch*(seed)).batch(batch_size)
    for inputs, outputs in train_ds:
      preds = mlp.forward(inputs)
      loss_batch = mlp.loss(preds, outputs)
      loss_total = loss_total + loss_batch
      #lt = lt + mlp.loss(preds, outputs)
      
      y_pred_argmax = np.argmax(preds, 1)

      y_true_argmax = np.argmax(outputs, 1)
      
      k = (y_pred_argmax == y_true_argmax)
      correct_preds_batch = len(k[k == True])
      correct_preds_total += correct_preds_batch

      mlp.backward(inputs, outputs)
    #print (correct_preds_total)
    #sys.exit()
    accuracy_epoch = correct_preds_total / X_train.shape[0]
    print('Number of Epoch = {} - Average CCE:= {} - Average Accuracy:= {}%'.format(epoch + 1, np.sum(loss_total) / X_train.shape[0], accuracy_epoch * 100))
    
  time_taken = time.time() - time_start
  print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

In [55]:
# Set number of epochs
NUM_EPOCHS = 10
NUM_EPOCHS = 20

In [74]:
#default training
mlp_default = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "default", lr = 0.05, device='gpu')
train_model(mlp_default, NUM_EPOCHS, seed, 1000,512)

  5%|▌         | 1/20 [00:02<00:50,  2.63s/it]

Number of Epoch = 1 - Average CCE:= 0.02209373372395833 - Average Accuracy:= 30.12666666666667%


 10%|█         | 2/20 [00:05<00:47,  2.62s/it]

Number of Epoch = 2 - Average CCE:= 0.02059483846028646 - Average Accuracy:= 35.03666666666667%


 15%|█▌        | 3/20 [00:07<00:44,  2.61s/it]

...

Number of Epoch = 18 - Average CCE:= 0.017302909342447916 - Average Accuracy:= 45.35333333333333%


 95%|█████████▌| 19/20 [00:49<00:02,  2.63s/it]

Number of Epoch = 19 - Average CCE:= 0.01727609659830729 - Average Accuracy:= 45.425%


100%|██████████| 20/20 [00:52<00:00,  2.63s/it]

Number of Epoch = 20 - Average CCE:= 0.01724445597330729 - Average Accuracy:= 45.565%

Total time taken (in seconds): 52.57


In [57]:
#l2reg training
mlp_l2 = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "L2", lambda_l2 = 0.05, device='gpu')
train_model(mlp_l2, NUM_EPOCHS, seed, 1000,512)

  5%|▌         | 1/20 [00:05<01:43,  5.45s/it]

Number of Epoch = 1 - Average CCE:= 0.020915535481770834 - Average Accuracy:= 33.78666666666666%


 10%|█         | 2/20 [00:09<01:18,  4.37s/it]

Number of Epoch = 2 - Average CCE:= 0.016667029825846353 - Average Accuracy:= 47.20666666666667%


 15%|█▌        | 3/20 [00:12<01:04,  3.77s/it]

...

Number of Epoch = 18 - Average CCE:= 0.014114368693033855 - Average Accuracy:= 55.51166666666667%


 95%|█████████▌| 19/20 [01:01<00:03,  3.09s/it]

Number of Epoch = 19 - Average CCE:= 0.01410935770670573 - Average Accuracy:= 55.601666666666674%


100%|██████████| 20/20 [01:04<00:00,  3.22s/it]

Number of Epoch = 20 - Average CCE:= 0.014083284505208333 - Average Accuracy:= 55.60000000000001%

Total time taken (in seconds): 64.45


In [60]:
#dropout training
mlp_dropout = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = "dropout", dropout = 0.3, device='gpu')
train_model(mlp_dropout, NUM_EPOCHS, seed, 1000,512)

  5%|▌         | 1/20 [00:05<01:43,  5.45s/it]

Number of Epoch = 1 - Average CCE:= 0.026429695638020834 - Average Accuracy:= 16.28333333333333%


 10%|█         | 2/20 [00:10<01:38,  5.45s/it]

Number of Epoch = 2 - Average CCE:= 0.02117415771484375 - Average Accuracy:= 32.62833333333333%


 15%|█▌        | 3/20 [00:21<02:12,  7.79s/it]

...

Number of Epoch = 18 - Average CCE:= 0.01068144022623698 - Average Accuracy:= 65.78666666666668%


 95%|█████████▌| 19/20 [01:05<00:02,  2.79s/it]

Number of Epoch = 19 - Average CCE:= 0.010597724405924479 - Average Accuracy:= 65.97666666666666%


100%|██████████| 20/20 [01:08<00:00,  3.44s/it]

Number of Epoch = 20 - Average CCE:= 0.010526710001627604 - Average Accuracy:= 66.24666666666667%

Total time taken (in seconds): 68.79


## One Step Inference

#function to test the model and return CCE and Accuracy

In [61]:
#function to test the model and return CCE and Accuracy
def test_model(mlp):
  test_loss_total = tf.Variable(0, dtype=tf.float32)
  correct_preds_total = 0
  #test_loss_total = 0.0
  for inputs, outputs in test_ds:
    preds = mlp.forward(inputs)
    test_loss_total = test_loss_total + mlp.loss(preds, outputs)
    y_pred_argmax = np.argmax(preds, 1)
    y_true_argmax = np.argmax(outputs, 1)
    k = (y_pred_argmax == y_true_argmax)
    correct_preds_batch = len(k[k == True])

    correct_preds_total += correct_preds_batch
  
  tot_cce = np.sum(test_loss_total.numpy()) / X_test.shape[0]
  tot_acc = correct_preds_total/X_test.shape[0]
  print('Test CCE: {:.4f}, Accuracy : {}'.format(tot_cce, tot_acc))
  return (tot_cce, tot_acc)

In [62]:
#testing default
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
test_model(mlp_default)

Test CCE: 0.0763, Accuracy : 0.5233


(0.07634425048828125, 0.5233)

In [63]:
#testing L2 model
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
test_model(mlp_l2)

Test CCE: 0.0877, Accuracy : 0.4578


(0.08767657470703125, 0.4578)

In [64]:
#testing dropout model
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
test_model(mlp_dropout)

Test CCE: 0.0543, Accuracy : 0.6577


(0.054267138671875, 0.6577)

In [65]:
# Obtaining inferences for all three models - 10 iteration per model
from tqdm import tqdm

############## Setting values as per dataset ##############
size_input = 784
size_hidden_1 = 100 
size_hidden_2 = 100
size_output = 10 
number_of_train_examples = X_train.shape[0]
number_of_test_examples = X_test.shape[0]
print (f"{number_of_test_examples},{number_of_train_examples}")

########################################################
'''
Additional steps done above
1. Reshape to flatten
2. Normalization
3. One hot encoding
'''

#######################################################
reg_type_list = ["default", "L2", "dropout"]
inference_stats_dict = {"default":{}, "L2": {}, "dropout": {}}
lambda_val = 0.05
dropout_val = 0.3
NUM_EPOCHS = 20
random_seeds = list(np.random.randint(low=4000, high=5000, size=10))
inf_iters = 10
device_type = 'gpu'
shuffle_size = 1000
batch_size = 512
learning_rate = 0.05
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)
####################################################

for reg_type in reg_type_list:
  print (f"## START: Current Model: {reg_type} ##")
  cce_list = []
  acc_list = []
  for iter in tqdm(range(inf_iters)):
    print(f"\n** Inference Iteration: {iter} **")
    cur_seed = random_seeds[iter]
    print (f"\n#Training {reg_type} model with seed {cur_seed}#")
    np.random.seed(cur_seed)
    tf.random.set_seed(cur_seed)

    # Model initialization as per regularization type - regularization code triggers only if knob is on
    mlp = MLP_combined(size_input, size_hidden_1, size_hidden_2, size_output, reg_mode = reg_type, lambda_l2 = lambda_val, dropout = dropout_val, lr = learning_rate, device=device_type)
    train_model(mlp, NUM_EPOCHS, cur_seed, shuffle_size, batch_size)

    # Testing the data
    (cce, acc) = test_model(mlp)
    print(f"seed: {cur_seed}, Cross Entropy loss: {cce}, Accuracy: {acc}")

    cce_list.append(cce)
    acc_list.append(acc)

  inference_stats_dict[reg_type]["cce_list"] = cce_list
  inference_stats_dict[reg_type]["acc_list"] = acc_list
  print (f"## END: Current Model: {reg_type}##")
  print (f"Current inference results: {inference_stats_dict}")
################################   

10000,60000
## START: Current Model: default ##


  0%|          | 0/10 [00:00<?, ?it/s]


** Inference Iteration: 0 **

#Training default model with seed 4225#



  5%|▌         | 1/20 [00:02<00:50,  2.64s/it]

Number of Epoch = 1 - Average CCE:= 0.02594561767578125 - Average Accuracy:= 18.14%


...

Number of Epoch = 19 - Average CCE:= 0.013116177368164063 - Average Accuracy:= 58.34166666666667%



100%|██████████| 20/20 [00:55<00:00,  2.79s/it]

Number of Epoch = 20 - Average CCE:= 0.012697203572591145 - Average Accuracy:= 59.660000000000004%

Total time taken (in seconds): 55.83



100%|██████████| 10/10 [09:30<00:00, 57.08s/it]

Test CCE: 0.0644, Accuracy : 0.5966
seed: 4912, Cross Entropy loss: 0.0644167724609375, Accuracy: 0.5966
## END: Current Model: dropout##
Current inference results: {'default': {'cce_list': [0.11522347412109375, 0.08467548828125, 0.08576741333007812, 0.1145487060546875, 0.08642153930664062, 0.09960307006835938, 0.07544818725585938, 0.104943408203125, 0.1073142578125, 0.11834266357421876], 'acc_list': [0.2849, 0.4742, 0.4646, 0.2888, 0.4625, 0.3818, 0.5296, 0.3482, 0.333, 0.2655]}, 'L2': {'cce_list': [0.11575765380859375, 0.08069005737304688, 0.09145341186523437, 0.11504547119140625, 0.07218370971679687, 0.10011902465820313, 0.08523603515625, 0.11567591552734376, 0.10225344848632813, 0.1164573974609375], 'acc_list': [0.2839, 0.5004, 0.4335, 0.2883, 0.5532, 0.3809, 0.4725, 0.2836, 0.3672, 0.2795]}, 'dropout': {'cce_list': [0.09012791137695313, 0.05209363403320313, 0.07554984130859375, 0.059787115478515625, 0.06057958984375, 0.07330875244140625, 0.07488440551757812, 0.06791614379882813, 0

In [66]:
print ("###### Final stats ##########")
print (inference_stats_dict)

###### Final stats ##########
{'default': {'cce_list': [0.11522347412109375, 0.08467548828125, 0.08576741333007812, 0.1145487060546875, 0.08642153930664062, 0.09960307006835938, 0.07544818725585938, 0.104943408203125, 0.1073142578125, 0.11834266357421876], 'acc_list': [0.2849, 0.4742, 0.4646, 0.2888, 0.4625, 0.3818, 0.5296, 0.3482, 0.333, 0.2655]}, 'L2': {'cce_list': [0.11575765380859375, 0.08069005737304688, 0.09145341186523437, 0.11504547119140625, 0.07218370971679687, 0.10011902465820313, 0.08523603515625, 0.11567591552734376, 0.10225344848632813, 0.1164573974609375], 'acc_list': [0.2839, 0.5004, 0.4335, 0.2883, 0.5532, 0.3809, 0.4725, 0.2836, 0.3672, 0.2795]}, 'dropout': {'cce_list': [0.09012791137695313, 0.05209363403320313, 0.07554984130859375, 0.059787115478515625, 0.06057958984375, 0.07330875244140625, 0.07488440551757812, 0.06791614379882813, 0.06906673583984375, 0.0644167724609375], 'acc_list': [0.4389, 0.6743, 0.529, 0.6259, 0.623, 0.5426, 0.5337, 0.5756, 0.5689, 0.5966]}}


In [68]:
import pandas as pd

for ele in inference_stats_dict:
  print (f"## Model: {ele} ###")
  print (f"Cross entropy loss across trials : {inference_stats_dict[ele]['cce_list']}")
  print (f"Accuracy across trials : {inference_stats_dict[ele]['acc_list']}")

## Model: default ###
Cross entropy loss across trials : [0.11522347412109375, 0.08467548828125, 0.08576741333007812, 0.1145487060546875, 0.08642153930664062, 0.09960307006835938, 0.07544818725585938, 0.104943408203125, 0.1073142578125, 0.11834266357421876]
Accuracy across trials : [0.2849, 0.4742, 0.4646, 0.2888, 0.4625, 0.3818, 0.5296, 0.3482, 0.333, 0.2655]
## Model: L2 ###
Cross entropy loss across trials : [0.11575765380859375, 0.08069005737304688, 0.09145341186523437, 0.11504547119140625, 0.07218370971679687, 0.10011902465820313, 0.08523603515625, 0.11567591552734376, 0.10225344848632813, 0.1164573974609375]
Accuracy across trials : [0.2839, 0.5004, 0.4335, 0.2883, 0.5532, 0.3809, 0.4725, 0.2836, 0.3672, 0.2795]
## Model: dropout ###
Cross entropy loss across trials : [0.09012791137695313, 0.05209363403320313, 0.07554984130859375, 0.059787115478515625, 0.06057958984375, 0.07330875244140625, 0.07488440551757812, 0.06791614379882813, 0.06906673583984375, 0.0644167724609375]
Accurac

In [69]:
import pandas as pd
df = pd.DataFrame()
df["seed"] = random_seeds
df["default_cce"] = inference_stats_dict["default"]["cce_list"]
df["L2_cce"] = inference_stats_dict["L2"]["cce_list"]
df["dropout_cce"] = inference_stats_dict["dropout"]["cce_list"]

df["default_acc"] = inference_stats_dict["default"]["acc_list"]
df["L2_acc"] = inference_stats_dict["L2"]["acc_list"]
df["dropout_acc"] = inference_stats_dict["dropout"]["acc_list"]

In [70]:
df

,seed,default_cce,L2_cce,dropout_cce,default_acc,L2_acc,dropout_acc
0,4225,0.115223,0.115758,0.090128,0.2849,0.2839,0.4389
1,4555,0.084675,0.080690,0.052094,0.4742,0.5004,0.6743
2,4193,0.085767,0.091453,0.075550,0.4646,0.4335,0.5290
3,4027,0.114549,0.115045,0.059787,0.2888,0.2883,0.6259
4,4747,0.086422,0.072184,0.060580,0.4625,0.5532,0.6230
5,4106,0.099603,0.100119,0.073309,0.3818,0.3809,0.5426
6,4079,0.075448,0.085236,0.074884,0.5296,0.4725,0.5337
7,4285,0.104943,0.115676,0.067916,0.3482,0.2836,0.5756
8,4648,0.107314,0.102253,0.069067,0.3330,0.3672,0.5689
9,4912,0.118343,0.116457,0.064417,0.2655,0.2795,0.5966


In [75]:
for ele in inference_stats_dict:
  print (f"## Model: {ele} ###")
  print (f"Mean of cross entropy : {np.mean(inference_stats_dict[ele]['cce_list'])}")
  print (f"Variance of cross entropy : {np.var(inference_stats_dict[ele]['cce_list'])}")
  print (f"Mean of accuracies : {np.mean(inference_stats_dict[ele]['acc_list'])}")
  print (f"Variance of accuracies : {np.var(inference_stats_dict[ele]['acc_list'])}")
  print ()

## Model: default ###
Mean of cross entropy : 0.09922882080078126
Variance of cross entropy : 0.00020765916375456756
Mean of accuracies : 0.38331000000000004
Variance of accuracies : 0.007880282899999999

## Model: L2 ###
Mean of cross entropy : 0.09948721252441406
Variance of cross entropy : 0.00024313515187576748
Mean of accuracies : 0.3843
Variance of accuracies : 0.009292995999999998

## Model: dropout ###
Mean of cross entropy : 0.06877309020996095
Variance of cross entropy : 0.00010057566869095305
Mean of accuracies : 0.5708499999999999
Variance of accuracies : 0.0038481864999999997

